# Sorting Algorithms

https://leetcode.com/problems/sort-an-array/

| Algorithm Name | Time Complexity | Space Complexity |
|----------------| -- | -- |
| Bubble Sort    | `O(n^2)` | `O(1)` |
| Selection Sort | `O(n^2)` | `O(1)` |
| Insertion Sort | `O(n^2)` | `O(1)` |
| Merge Sort     | `O(n log n)` | `O(n)` |
| Heap Sort      | `O(n log n)` | `O(1)` if in-place, else `O(n)` |
| Quick Sort     | `O(n log n)` avg, `O(n^2)` worse | `O(n)` stack space |
| Bucket Sort    | `O(n)` | `O(k)`, where `k` is max element (positives only) |
| Count Sort     | `O(k + n)`, where `k` is difference between largest and smallest | `O(n)` |

## Bubble Sort

- Time: `O(n^2)`
- Space: `O(1)`

In [1]:
def bubble_sort(nums):
    while True:
        swapped = False

        for i in range(1, len(nums)):
            # if element is smaller than previous, it is not in ascending order
            if nums[i-1] > nums[i]:
                # swap elements, mark that an out of order element was found
                nums[i], nums[i-1] = nums[i-1], nums[i]
                swapped = True

        # if no swaps (out of order elements) then the list is in ascending order, finished
        if not swapped:
            break

    return nums

## Selection Sort
- Time: `O(n^2)`
- Space: `O(1)`

In [2]:
def selection_sort(nums):
    # can skip last iteration since it will be in correct place
    for i in range(len(nums) - 1):
        min_index = i

        # for each sub-array, find smallest element
        for j in range(i+1, len(nums)):
            if nums[j] < nums[min_index]:
                min_index = j

        # swap smallest element with beginning element (i), continue to next iteration (i+1)
        # do not need to swap if beginning element was already smallest (already correct place)
        if min_index != i:
            nums[i], nums[min_index] = nums[min_index], nums[i]

    return nums

## Insertion Sort
- Time: `O(n^2)`
- Space: `O(1)`

In [3]:
def insertion_sort(nums):
    # go through each element skipping first
    for i in range(1, len(nums)):
        j = i

        # shift element at i down repeatedly (j) until it is not smaller than previous
        while j > 0 and nums[j-1] > nums[j]:
            nums[j-1], nums[j] = nums[j], nums[j-1]
            j -= 1

    return nums

## Merge Sort
- Time: `O(n log n)`
- Space: `O(n)`

### _merge()

The helper function `_merge()` merges two sorted lists into a single sorted list

Same result as `heapq.merge()`

In [4]:
def _merge(a, b):
    merged = []

    # add a and b's elements to empty list in order
    i = 0
    j = 0
    while (i != len(a) and j != len(b)):
        if a[i] <= b[j]:
            merged.append(a[i])
            i += 1
        else:
            merged.append(b[j])
            j += 1

    # handle remaining elements once one list finished
    while i != len(a):
        merged.append(a[i])
        i += 1

    while j != len(b):
        merged.append(b[j])
        j += 1

    return merged

### Top Down Merge Sort

This approach uses recursion, splitting the list in half until it reaches 1 element size arrays to begin merging

Uses the `_merge()` helper function

In [5]:
def top_down_merge_sort(nums):
    # base case, return nums if size 1 or 0
    if not len(nums) > 1:
        return nums

    # split into 2 lists
    midpoint = len(nums) // 2
    left = nums[:midpoint]   # [:k] exclusive of k
    right = nums[midpoint:]  # [k:] inclusive of k

    left_sorted = top_down_merge_sort(left)
    right_sorted = top_down_merge_sort(right)

    # merge two resulting
    return _merge(left_sorted, right_sorted)


### Bottom Up Merge Sort

This is an iterative approach, that starts at size 1 array and doubles at each iteration while merging until input size reached

Uses the `_merge()` helper function

In [6]:
def bottom_up_merge_sort(nums):
    n = len(nums)
    result = list(nums)
    width = 1

    while width < n:
        for start in range(0, n, 2 * width):
            mid = min(start + width, n)
            end = min(start + 2 * width, n)

            merged = _merge(result[start:mid], result[mid:end])
            result[start:end] = merged

        width *= 2

    return result

### Bottom Up Merge Sort with one buffer array

This is the same iterative approach, but does not use slicing and maintains a single auxilary buffer

In [7]:
def bottom_up_merge_sort_less_space(nums):
    n = len(nums)

    # maintain running merged result and a buffer to use during element swaps
    merged = list(nums)
    buffer = [None] * n

    width = 1
    while width < n:
        # width will start at 1 and go to 2, 4, 8, ..., n
        for start in range(0, n, width * 2):
            # divide into 2 halves, use min to avoid out of index
            mid = min(start + width, n)
            end = min(start + 2 * width, n)

            # treat as left and right partition
            left = start
            right = mid

            # copy in order to auxiliary array
            i = start
            while i < end:
                # if left still has elements (hasnt gone past midpoint, left's end)
                #    and either right is out of elements (past end)
                #            or left <= right (smaller)
                # then use element from left
                if left < mid and (right >= end or merged[left] <= merged[right]):
                    buffer[i] = merged[left]
                    left += 1

                # else use element from right
                else:
                    buffer[i] = merged[right]
                    right += 1

                i += 1

        # swap buffer with merged result
        merged, buffer = buffer, merged
        width *= 2

    return merged

## Heap Sort

- Time: `O(n log n)`
- Space: `O(1)` if in place heap, else `O(n)` 

In [8]:
import heapq  # python's heap structure

def heap_sort_built_in(nums):
    # O(n) space
    heap = []
    for num in nums:
        heapq.heappush(heap, num)
    
    sorted_nums = []
    while len(heap) > 0:
        sorted_nums.append(heapq.heappop(heap))

    return sorted_nums

## Quick Sort

- Time: `O(n log n)` average, `O(n^2)` worst case (if sorted)
- Space: `O(n)` stack space

### Partition

Partition a subarray around a chosen pivot, so that all elements to the left of that pivot's final place are `<=` to it, and all elements to the right are `>=`

#### Lomuto's

The pivot is chosen to be the last element in the subarray

Two points are used:

- `i` tracks the boundary of elements less than the pivot

- `j` scans the array from `start` to `end - 1`

If `nums[j] < pivot`, increment `i` and swap `nums[i]` with `nums[j]`

After the scan, swap the `pivot` with `nums[i + 1]` (because `nums[0]` to `nums[i]` is now < `pivot`)

The `pivot` is now at `i + 1`, with all smaller elements to the left and larger to the right

In [9]:
def _partition_lomutos(nums, start, end):
    pivot = nums[end]  # start with last element

    # boundary of elements <= pivot
    i = start - 1

    # j scans subarray 
    for j in range(start, end):

        # if current element belongs in the left partition (<= pivot)
        if nums[j] <= pivot:
            # add slot to left partition
            i += 1
            # move it to the end of the left partition by swapping
            nums[i], nums[j] = nums[j], nums[i]

    # pivot was at end, place after last element in left partition by swapping
    nums[i+1], nums[end] = nums[end], nums[i+1]

    # return final pivot position
    return i + 1

#### Hoare's

The `pivot` is chosen to be the first element in the subarray

Two pointers, one from the start `i` and one from the end `j`, travel towards eachother

- everything to the left of `j` is `<= pivot`
  
- everything to the right of `i` is `>= pivot`

As long as `i < j`, when a pair of out-of-place elements is found, `nums[i]` and `nums[j]` are swapped

When the pointers cross, `j` is the final boundary between the two halves

In [10]:
def _partition_hoares(nums, start, end):
    pivot = nums[start]  # start with first element as pivot

    # initialize pointers just outside subarray
    i = start - 1 
    j = end + 1 

    while True:

        # bump i until num[i] is >= pivot
        i += 1
        while nums[i] < pivot:
            i += 1

        # decrease j until num[j] is <= pivot
        j -= 1
        while nums[j] > pivot:
            j -= 1

        # if pointers have met or crossed, partitioning is complete
        if i >= j:
            return j

        # otherwise, swap the two out-of-place elements since either:
        #   - nums[i] is >= pivot but on the left side
        #   - nums[j] is <= pivot but on the right side
        nums[i], nums[j] = nums[j], nums[i]

### Sort

1. Partition the subarray from `start` to `end` so that all elements to left of index `p` are less than that element, and all elements to right are greater
2. Recursively run `quick_sort` on the two partition halves until `nums` is sorted

In [11]:
def quick_sort_hoares(nums):
    def _quick_sort_recursive(start, end):
        if start < end:
            pivot = _partition_hoares(nums, start, end)
            _quick_sort_recursive(start, pivot)  # _partition_hoares returns the last index of the left-hand subarray (not necessarily where the pivot is)
            _quick_sort_recursive(pivot + 1, end)

    # sort entire given array
    _quick_sort_recursive(0, len(nums) - 1)
    return nums
    
def quick_sort_lomutos(nums):
    def _quick_sort_recursive(start, end):
        if start < end:
            pivot = _partition_lomutos(nums, start, end)
            _quick_sort_recursive(start, pivot - 1)  # _partition_lomutos returns the exact final pivot index
            _quick_sort_recursive(pivot + 1, end)

    _quick_sort_recursive(0, len(nums) - 1)
    return nums

def quick_sort(nums):
    return quick_sort_hoares(nums)  # better partitioning scheme

## Bucket Sort

- Time: `O(n)`
- Space: `O(k)`, where k is highest key

Only works for positive, though can be adjusted to include a negatives bucket which is prepended at end

In [12]:
def bucket_sort(nums):
    """
    Only works if nums are positive
    """

    # uses a lot of space
    buckets = [0] * (max(nums) + 1)
    
    for num in nums:
        buckets[num] += 1

    sorted_nums = []
    for (i, count) in enumerate(buckets):
        if count != 0:
            sorted_nums.extend([i]*count)

    return sorted_nums

## Count Sort
- Time: `O(k + n)`, where `k` is difference between largest and smallest value
- Space: `O(n)`

In [13]:
from collections import defaultdict

def count_sort(nums):
    counts = defaultdict(int)

    start = nums[0]
    end = nums[0]
    
    for num in nums:
        counts[num] += 1
        start = min(start, num)
        end = max(end, num)

    sorted_nums = []
    for i in range(start, end+1):
        if counts[i] >= 0:
            sorted_nums.extend([i]*counts[i])

    return sorted_nums